This notebook was inspired by this Anthropic's blog post:
- https://www.anthropic.com/research/building-effective-agents

and this source code:
- https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents/basic_workflows.ipynb

In [1]:
from openai import OpenAI
client = OpenAI()

In [2]:
def llm_call(prompt_question):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def chain(input, prompts):
    result = input
    for i, prompt in prompts:
        print(f"Step: {i}")
        result = llm_call(f"{prompt}: \n\n{result}")
        print(result)
    
    return result

In [2]:
# Example prompts for a multi-step analysis chain
analysis_prompts = [
    (1, "Analyze this text and identify the main themes and topics"),
    (2, "For each theme identified, provide specific examples from the text"),
    (3, "Synthesize the themes and examples into a coherent summary"),
    (4, "Generate 3 discussion questions based on the analysis")
]

# Sample input text
sample_text = """
The rise of artificial intelligence has transformed multiple industries in the past decade. 
Healthcare has seen improvements in diagnosis accuracy and treatment planning. 
Manufacturing has become more efficient with predictive maintenance and automated quality control.
However, these advancements also raise important ethical questions about privacy, job displacement, 
and the role of human decision-making in an increasingly automated world.
"""

# Run the chain
print("Running analysis chain...")
final_result = chain(sample_text, analysis_prompts)
print(final_result)

Running analysis chain...
Step: 1
The text conveys several main themes and topics related to the rise of artificial intelligence (AI). Here are the key points identified:

1. **Impact of AI on Industries**: The text highlights how AI has transformed various sectors, specifically mentioning:
   - **Healthcare**: Improvements in diagnostic accuracy and treatment planning.
   - **Manufacturing**: Increased efficiency through predictive maintenance and automated quality control.

2. **Ethical Considerations**: The advancements associated with AI raise significant ethical questions, including:
   - **Privacy**: Concerns about how data is collected, stored, and used.
   - **Job Displacement**: The impact of automation on employment and workforce dynamics.
   - **Human Decision-Making**: The role and importance of human involvement in decision-making processes as machines become more capable.

Overall, the text discusses both the positive transformations brought about by AI and the associated

In [3]:
from concurrent.futures import ThreadPoolExecutor

In [7]:
def parallel(prompt, inputs, n_workers=3):
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(llm_call, f"{prompt}: \n {input}") for input in inputs]
    
    return [f.result() for f in futures]


prompt = "Summarize this paragraph in one sentence"
inputs = [
    """
    Climate change is causing rising sea levels and extreme weather patterns globally. 
    Scientists have observed accelerating ice melt in polar regions, leading to coastal 
    flooding in low-lying areas. Additionally, communities worldwide are experiencing 
    more frequent and severe hurricanes, droughts, and other extreme weather events. 
    These changes pose significant risks to agriculture, infrastructure, and human 
    populations, particularly in vulnerable coastal regions and developing nations.
    """,
    """
    The development of quantum computers represents a major technological breakthrough
    with far-reaching implications. These powerful machines leverage quantum mechanical
    properties to perform complex calculations exponentially faster than classical computers.
    In the field of cryptography, quantum computers could break many current encryption
    methods while enabling new unbreakable encryption protocols. For drug discovery,
    quantum computers could simulate molecular interactions with unprecedented accuracy,
    potentially accelerating the development of new medicines and treatments for diseases.
    """,
    """
    Social media platforms have transformed how people communicate and share information
    in the modern digital age. Platforms like Facebook, Twitter, and Instagram have created
    virtual communities where users can instantly connect with friends and family across the
    globe. These platforms enable the rapid spread of news, ideas, and cultural trends,
    while also raising concerns about privacy, misinformation, and the impact on mental
    health. The rise of social media has also revolutionized marketing, activism, and
    how businesses engage with their customers.
    """
]

parallel(prompt, inputs)

['Climate change is resulting in rising sea levels and increased extreme weather events, threatening agriculture, infrastructure, and communities, especially in coastal and developing areas.',
 'Quantum computers are a significant technological advancement that can perform complex calculations much faster than classical computers, with transformative effects on cryptography and drug discovery.',
 'Social media platforms have revolutionized communication and information sharing, fostering global connections while also introducing concerns about privacy, misinformation, and mental health, and transforming marketing and activism.']

In [10]:
from enum import Enum
from pydantic import BaseModel, Field 
from typing import List


class ProfileRouter(BaseModel):
    selected_profile: str = Field(description="You return one of 3 profiles: [hr, software engineer, product manager]")
    

def llm_call_route(problem):
    '''Selects which profile to use to analyse the input.'''
    response = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=[{'role': 'system', 'content': """
                   You select which of the following profiles should be used to solve a problem given as input:
                   1. hr
                   2. software engineer
                   3. product manager
                   Your output is solely the choosen profile string.
                   """},
                  {'role': 'user', 'content': f"We had this problem: \n {problem}. Output the appropriate profile to solve it:\n"}],
        response_format=ProfileRouter
    )
    return response.choices[0].message.parsed.selected_profile
    

llm_call_route("I need to fix this bug in our codebase")

'software engineer'

In [8]:
from typing import Dict


def router(input, routes: Dict[str, str]):
    
    route_response = llm_call_route(input)
    
    selected_prompt = routes[route_response]
    
    return llm_call(selected_prompt + f"Input: {input}")       

In [11]:
routes_profiles = {
    'hr': """You are an experienced HR professional focused on employee relations, recruitment, and workplace policies.
    Your expertise includes conflict resolution, talent acquisition, performance management, and ensuring compliance with labor laws.
    Analyze the input from an HR perspective and provide appropriate guidance.""",
    
    'software engineer': """You are a senior software engineer with deep technical expertise in software development, architecture, and best practices.
    Your skills include debugging, code optimization, system design, and technical problem-solving.
    Analyze the input from an engineering perspective and provide technical solutions.""",
    
    'product manager': """You are a seasoned product manager skilled in product strategy, user experience, and market analysis.
    Your expertise includes feature prioritization, roadmap planning, stakeholder management, and data-driven decision making.
    Analyze the input from a product perspective and provide strategic recommendations."""
}

problems = [
    "We need to improve our onboarding process for new employees to increase retention",
    "The authentication service is throwing intermittent 500 errors in production",
    "Our user engagement metrics have dropped 20% since the last release"
]

# Test the router with each problem
for problem in problems:
    print(f"\nProblem: {problem}")
    print(f"Response: {router(problem, routes_profiles)}")


Problem: We need to improve our onboarding process for new employees to increase retention
Response: hr

Problem: The authentication service is throwing intermittent 500 errors in production
Response: software engineer

Problem: Our user engagement metrics have dropped 20% since the last release
Response: product manager
